## Финальное соревнование

##### Автор: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), @neychev

In [11]:
import statistics

l = (52, 49, 31, 53, 44, 36, 50, 50, 43, 44, 45, 43, 52, 48, 42)
import seaborn as sns
import matplotlib.pyplot as plt
#sns.boxplot(l)
#plt.grid()

#median_value = statistics.median(l)

#print(median_value)

AttributeError: 'tuple' object has no attribute 'plot'

В данном задании вас ждет неизвестная зависимость. Ваша основная задача: **построить две лучших модели**, минимизирующих среднеквадратичную ошибку (MSE):
1. На первую модель не налагается ограничений.
2. Вторая модель должна быть **линейной**, т.е. представлять собой линейную комбинацию признаков плюс свободный член: $\boldsymbol{w}^{\top}\boldsymbol{x} + b$. При этом __вы можете использовать базовые математические операции для преобразования признаков__: np.exp, np.log, np.pow (полный список доступен в [документации](https://numpy.org/doc/stable/reference/routines.math.html)), а также линейные операции над ними (сумма, умножение на число и пр.). Для преобразования признаков вам будет необходимо написать функцию `my_transformation`. __Кол-во параметров (весов) используемых второй моделью не должно превышать 15 (включая свободный член).__

Настоятельно рекомендуем написать код "с нуля", лишь поглядывая на готовые примеры, а не просто "скопировать-вставить". Это поможет вам в будущем писать код более уверенно

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import os
import json

import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from catboost import CatBoostRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import warnings
from IPython.display import display, HTML, Javascript
display(HTML("<style>.container { width:70% !important; }</style>"))
pd.options.display.max_columns = None
warnings.filterwarnings('ignore') 
%autosave 60

Autosaving every 60 seconds


Загрузка данных происходит ниже. Если она не срабатывает, самостоятельно скачайте файл `hw_final_open_data.npy` и положите его в ту же директорию, что и ноутбук.

In [2]:
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_data.npy -O hw_final_open_data.npy
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_open_target.npy -O hw_final_open_data.npy

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [18]:
data = np.load(r"C:\Users\123123\Downloads\Тренировки ML\Финальное соревнование\hw_final_open_data.npy")
df = pd.DataFrame(data)

targ = np.load(r"C:\Users\123123\Downloads\Тренировки ML\Финальное соревнование\hw_final_open_target.npy")
target = pd.DataFrame(targ)
target.rename(columns={0:'target'}, inplace=True)

test_1 = np.load(r"C:\Users\123123\Downloads\Тренировки ML\Финальное соревнование\hw_final_closed_data.npy")
test = pd.DataFrame(test_1)

for column in train.columns:
    if column != 'target':  
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x=train[column], y=train['target'])
        plt.title(f'Scatterplot for {column} vs target')
        plt.xlabel(column)
        plt.ylabel('target')
        plt.show()

for column in train.columns:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=train[column])
    plt.title(f'Boxplot for {column}')
    plt.xlabel(column)
    plt.show()

for column in train.columns:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=test[column])
    plt.title(f'Boxplot for {column}')
    plt.xlabel(column)
    plt.show()

In [19]:
train = pd.concat([df, target], axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       800 non-null    float64
 1   1       800 non-null    float64
 2   2       800 non-null    float64
 3   3       800 non-null    float64
 4   4       800 non-null    float64
 5   5       800 non-null    float64
 6   6       800 non-null    float64
 7   target  800 non-null    float64
dtypes: float64(8)
memory usage: 50.1 KB


In [20]:
def preprocessing(data):
    data[8] = data[0] + data[1]
    data[9] = data[1] + data[2] + data[3] + data[4] + data[5] + data[6]
    return data

In [21]:
df = preprocessing(df)
df

,0,1,2,3,4,5,6,8,9
0,-0.565814,-0.430544,-0.56863,-0.379486,-0.527420,-0.160691,-0.575438,-0.996358,-2.642209
1,-0.562880,-0.051177,-0.56863,-0.134094,-0.506339,0.795601,-0.575438,-0.614058,-1.040076
2,-0.560292,0.987380,-0.56863,0.344372,-0.481199,3.249530,-0.575438,0.427087,2.956014
3,-0.561336,0.434373,-0.56863,0.107739,-0.492241,1.960089,-0.575438,-0.126963,0.865892
4,-0.563982,-0.249578,-0.56863,-0.251161,-0.515020,0.304072,-0.575438,-0.813561,-1.855756
...,...,...,...,...,...,...,...,...,...
795,-0.562995,-0.076328,-0.56863,-0.148095,-0.507292,0.733981,-0.575438,-0.639323,-1.141801
796,-0.566049,-0.445115,-0.56863,-0.391541,-0.528851,-0.199316,-0.575438,-1.011164,-2.708890
797,-0.560825,0.675599,-0.56863,0.214591,-0.487007,2.526183,-0.575438,0.114774,1.785298
798,-0.561542,0.350201,-0.56863,0.068784,-0.494273,1.760945,-0.575438,-0.211341,0.541589


In [22]:
#X = train.drop(columns='target')  
#y = train['target']  

X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.2, random_state=42)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       800 non-null    float64
 1   1       800 non-null    float64
 2   2       800 non-null    float64
 3   3       800 non-null    float64
 4   4       800 non-null    float64
 5   5       800 non-null    float64
 6   6       800 non-null    float64
 7   8       800 non-null    float64
 8   9       800 non-null    float64
dtypes: float64(9)
memory usage: 56.4 KB


In [24]:
cat_col = train.select_dtypes(include=['object']).columns.to_list()
models = [CatBoostRegressor(n_estimators=100, verbose=10, random_seed=42, cat_features=cat_col),
          Ridge(),
          LinearRegression(),
          Lasso()]

for model in models:
    model.fit(X_train, y_train)
    y_prediction = model.predict(X_test)
    mse = mean_squared_error(y_test, y_prediction)
    print(f'MSE для {model.__class__.__name__}: {mse}')

Learning rate set to 0.24806
0:	learn: 1.6814888	total: 1.67ms	remaining: 165ms
10:	learn: 0.3312356	total: 17.1ms	remaining: 138ms
20:	learn: 0.2981454	total: 32ms	remaining: 120ms
30:	learn: 0.2922838	total: 46.5ms	remaining: 103ms
40:	learn: 0.2849338	total: 60.9ms	remaining: 87.6ms
50:	learn: 0.2796721	total: 75ms	remaining: 72.1ms
60:	learn: 0.2766406	total: 88.1ms	remaining: 56.3ms
70:	learn: 0.2724939	total: 101ms	remaining: 41.4ms
80:	learn: 0.2687567	total: 113ms	remaining: 26.6ms
90:	learn: 0.2656014	total: 125ms	remaining: 12.4ms
99:	learn: 0.2638261	total: 137ms	remaining: 0us
MSE для CatBoostRegressor: 0.10220842739386234
MSE для Ridge: 0.09489847016066782
MSE для LinearRegression: 0.09551844682324234
MSE для Lasso: 0.38385265381475986


In [49]:
assert os.path.exists('hw_final_open_data.npy'), 'Please, download `hw_final_open_data.npy` and place it in the working directory'
assert os.path.exists('hw_final_open_target.npy'), 'Please, download `hw_final_open_target.npy` and place it in the working directory'
data = np.load('hw_final_open_data.npy', allow_pickle=False)
target = np.load('hw_final_open_target.npy', allow_pickle=False)

Разбивка на `train` и `val` опциональна и сделана для вашего удобства.

In [50]:
train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.3)

### Модель №1
Напоминаем, в первой части задания ваша основная задача – получить наилучший результат без ограничений на модель. Сдаваться будут только предсказания модели.

Пример с использованием Random Forest доступен ниже.

In [27]:
rf = Ridge()
rf.fit(train_x, train_y)

print(
    f'train mse =\t {mean_squared_error(np.round(rf.predict(train_x), 2), np.round(train_y)):.5f}',
    f'validation mse = {mean_squared_error(np.round(rf.predict(valid_x)), np.round(valid_y)):.5f}',
    sep='\n'
)

train mse =	 0.18842
validation mse = 0.24583


##### Сдача первой части соревнования
Загрузите файл `hw_final_closed_data.npy` (ссылка есть на странице с заданием). Если вы используете sklearn-совместимую модель, для генерации посылки вы можете воспользоваться функцией `get_predictions`. В ином случае перепишите функцию для вашей модели и запустите код под следующей ячейкой для генерации посылки.

In [28]:
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/23f_yandex_ml_trainings/homeworks/assignment_final/hw_final_closed_data.npy -O hw_final_closed_data.npy

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [59]:
assert os.path.exists('hw_final_closed_data.npy'), 'Please, download `hw_final_closed_data.npy` and place it in the working directory'
closed_data = np.load('hw_final_closed_data.npy', allow_pickle=False)
#closed_data = preprocessing(closed_data)
#closed_data = pd.DataFrame(closed_data)

In [35]:
closed_data = preprocessing(closed_data)
closed_data

,0,1,2,3,4,5,6,8,9
0,-0.559878,1.281356,-0.56863,0.460149,-0.476420,3.924734,-0.575438,0.721478,4.045751
1,-0.561259,0.467296,-0.56863,0.122720,-0.491478,2.037739,-0.575438,-0.093963,0.992209
2,-0.564778,-0.345467,-0.56863,-0.315276,-0.520689,0.060612,-0.575438,-0.910245,-2.264887
3,-0.559726,1.401775,-0.56863,0.506055,-0.474613,4.199694,-0.575438,0.842049,4.488843
4,-0.559901,1.263218,-0.56863,0.453163,-0.476699,3.883242,-0.575438,0.703317,3.978856
...,...,...,...,...,...,...,...,...,...
195,-0.562022,0.180290,-0.56863,-0.013141,-0.498809,1.355861,-0.575438,-0.381732,-0.119867
196,-0.566792,-0.482617,-0.56863,-0.424728,-0.533172,-0.300128,-0.575438,-1.049409,-2.884714
197,-0.560755,0.712904,-0.56863,0.230566,-0.486263,2.613184,-0.575438,0.152149,1.926323
198,-0.560232,1.027075,-0.56863,0.360342,-0.480519,3.341058,-0.575438,0.466843,3.103888


Если необходимо, преобразуйте данные. Преобразованную матрицу объект-признак сохраните в переменную `closed_data`.

In [36]:
# optional transformations

In [45]:
def get_predictions(model, eval_data, step=10):
    predicted_values = model.predict(eval_data)
    return predicted_values

Обращаем ваше внимание, предсказания округляются до сотых!

In [38]:
predicted_values = np.round(get_predictions(model=rf, eval_data=closed_data), 2)

assert predicted_values.shape == (closed_data.shape[0], ) # predictions should be just one-dimensional array

In [39]:
# do not change the code in the block below
# __________start of block__________
def float_list_to_comma_separated_str(_list):
    _list = list(np.round(np.array(_list), 2))
    return ','.join([str(x) for x in _list])

submission_dict = {
    'predictions': float_list_to_comma_separated_str(predicted_values)
}
with open('submission_dict_final_p01.json', 'w') as iofile:
    json.dump(submission_dict, iofile)
    
print('File saved to `submission_dict_final_p01.npy`')
# __________end of block__________

File saved to `submission_dict_final_p01.npy`


### Модель №2
Функция `my_transformation` принимает на вход матрицу объект-признак (`numpy.ndarray` типа `np.float`) и преобразует ее в новую матрицу. Данная функция может использовать только numpy-операции, а также арифметические действия.

Для примера доступна функция ниже. Она лишь добавляет новый признак, представляющий собой произведение первого и второго исходных признаков (считая с нуля).

In [66]:
train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.3)

In [67]:
def my_transformation(feature_matrix: np.ndarray):
    new_feature_matrix = np.zeros((feature_matrix.shape[0], feature_matrix.shape[1]+1))
    new_feature_matrix[:, :feature_matrix.shape[1]] = feature_matrix
    new_feature_matrix[:, -1] = feature_matrix[:, 0] * feature_matrix[:, 1]
    new_feature_matrix[:, -2] = (feature_matrix[:, 0] * feature_matrix[:, 1] + feature_matrix[:, 2] * feature_matrix[:, 3]
                                 + feature_matrix[:, 4] * feature_matrix[:, 5] + feature_matrix[:, 6])
    return new_feature_matrix

In [68]:
transformed_train_x = my_transformation(train_x)

In [69]:
lr = Ridge()
lr.fit(transformed_train_x, train_y)

print(
    f'train mse =\t {mean_squared_error(lr.predict(transformed_train_x), train_y):.5f}',
    f'validation mse = {mean_squared_error(lr.predict(my_transformation(valid_x)), valid_y):.5f}',
    sep='\n'
)

train mse =	 0.08883
validation mse = 0.09272


Обращаем ваше внимание, что параметры линейной модели будут округляться до __четырех знаков после запятой__. Это не должно сильно повлиять на качество предсказаний:

In [70]:
original_predictions = lr.predict(transformed_train_x)
rounded_predictions = transformed_train_x.dot(np.round(lr.coef_, 4)) + np.round(lr.intercept_, 4)


assert np.allclose(original_predictions, rounded_predictions, atol=1e-3)

Параметры вашей модели:

In [71]:
w_list = list(np.round(lr.coef_, 4))
print(f'w = {list(np.round(lr.coef_, 4))}\nb = {np.round(lr.intercept_, 4)}')

w = [0.0084, 0.2445, 0.0, 0.3886, 0.0525, 0.8121, -0.8893, -0.1457]
b = 1.118


Напоминаем, ваша модель не должна использовать более 15 параметров (14 весов плюс свободный член).

In [72]:
assert len(w_list) + 1 <= 15

##### Сдача второй части соревнования
Для сдачи вам достаточно отправить функцию `my_transformation` и параметры вашей модели в контест в задачу №2. Пример посылки доступен ниже

In [57]:
# __________example_submission_start__________

def my_transformation(feature_matrix: np.ndarray):
    new_feature_matrix = np.zeros((feature_matrix.shape[0], feature_matrix.shape[1]+1))
    new_feature_matrix[:, :feature_matrix.shape[1]] = feature_matrix
    new_feature_matrix[:, -1] = feature_matrix[:, 0
    ] * feature_matrix[:, 1]
    return new_feature_matrix

w_submission = [-0.0027, -0.2637, 0.0, -0.1134, -0.0165, -0.9329, 0.0, 0.1293]
b_submission = 1.1312
# __________example_submission_end__________

На этом задание завершено. Поздравляем!